In [4]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [4]:
#importing packages
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests                                      # reading data
from io import StringIO

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import zero_one_loss as J01
from sklearn.preprocessing import OneHotEncoder
import xgboost
from sklearn.linear_model import LinearRegression

import sklearn.tree as tree

import seaborn as sns

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)  

In [5]:
# Reading in the CSV file
Xtr = pd.read_csv("data/X_TRAINING_SET_Diabetes_ENCODED.csv")
Ytr = pd.read_csv("data/Y_TRAINING_SET_EDIT.csv")
                  
Xtest = pd.read_csv("data/X_VALIDATION_SET_ENCODED.csv")
Ytest = pd.read_csv("data/Y_VALIDATION_SET_EDIT.csv")



In [20]:
'''
Ytr['readmitted'] = Ytr['readmitted'].map({'<30': 1, 'NO': 0, '>30': 2})
Ytr.to_csv('data/Y_TRAINING_SET_EDIT.csv', index=False)

Ytest['readmitted'] = Ytest ['readmitted'].map({'<30': 1, 'NO': 0, '>30': 2})
Ytest.to_csv('data/Y_VALIDATION_SET_EDIT.csv', index=False)
'''

Understanding the data, the correlations and understanding why it performs well for readmission = No and very poorly for readmissions <30 

In [8]:
d = pd.DataFrame(Xtr)
d = d.drop(columns=['Unnamed: 0', 'index'])
d

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
0,146752038.0,25174899.0,1.0,1.0,7.0,3.0,37.0,1.0,13.0,0.0,...,1,0,1,0,1,0,1,0,0,1
1,83354610.0,298521.0,1.0,14.0,7.0,3.0,57.0,0.0,10.0,0.0,...,1,0,1,0,1,0,0,1,1,0
2,170636856.0,77827617.0,2.0,6.0,1.0,4.0,71.0,0.0,11.0,0.0,...,1,0,1,0,1,0,0,1,0,1
3,221787444.0,38516562.0,1.0,1.0,7.0,1.0,29.0,0.0,6.0,0.0,...,1,0,1,0,1,0,0,1,1,0
4,163020096.0,46248840.0,1.0,1.0,7.0,10.0,49.0,1.0,18.0,0.0,...,1,0,1,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,289025640.0,50693553.0,1.0,6.0,7.0,9.0,72.0,4.0,22.0,0.0,...,1,0,1,0,1,0,1,0,0,1
71232,169681170.0,66287709.0,5.0,1.0,1.0,3.0,1.0,3.0,9.0,1.0,...,1,0,1,0,1,0,0,1,0,1
71233,106962006.0,24745671.0,2.0,1.0,4.0,5.0,14.0,4.0,23.0,0.0,...,1,0,1,0,1,0,0,1,0,1
71234,164976492.0,59602239.0,1.0,1.0,7.0,3.0,45.0,6.0,13.0,0.0,...,1,0,1,0,1,0,0,1,1,0


In [10]:
d = pd.concat([d,Ytr])
d

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,Unnamed: 0,readmitted
0,146752038.0,25174899.0,1.0,1.0,7.0,3.0,37.0,1.0,13.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN
1,83354610.0,298521.0,1.0,14.0,7.0,3.0,57.0,0.0,10.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN
2,170636856.0,77827617.0,2.0,6.0,1.0,4.0,71.0,0.0,11.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN
3,221787444.0,38516562.0,1.0,1.0,7.0,1.0,29.0,0.0,6.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN
4,163020096.0,46248840.0,1.0,1.0,7.0,10.0,49.0,1.0,18.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89460.0,0.0
71232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60620.0,0.0
71233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34086.0,0.0
71234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58067.0,2.0


In [22]:

# Compute the correlation matrix
corr = d.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})